In [2]:
#__init__.py conversion

from .utils import featureless_random_graph

ImportError: attempted relative import with no known parent package

In [4]:
#layers.py conversion

import torch
import torch.nn as nn 
import torch.nn.functional as F
import random 
from torch.nn import init
from utils import featureless_random_graph,get_feature_func,get_neigh_list
"""
Here we will implement the GraphSAGE layer and a graph convolutional layer.
                                                 ^^^^^^^^^^^^^^^^^^^^^^^^^ TODO
Recommend reading 
The original GraphSAGE paper:
https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf

Influenced heavily by https://github.com/williamleif/graphsage-simple/

TERMINOLOGY:
    A graph is described by a set G = {V,E} of nodes and edges.
    A representation of a node is any feature associated to that node. I avoid
    using 'feature' generally here as this definition also applies to neural
    network produced features of nodes ie. trainable embeddings.
    - In this case the representation is initially the node features (PCA) for
      layer one, and for later layers is the output of the previous layer.

We implement a GraphSAGE layer as 2 distinct objects:
1. A layer-like Encoder class, with an associated weight matrix W and activation
   function. This level takes a representation of a node and combines it with
   another representation generated by the Aggregator.
2. An Aggregator class which generates a representation of a node from it's
   neighbours representations in the previous layer. Can be seen as a learnt
   representation producing function which can take some complex-NN forms if needed.
This two object layout is useful as the aggregator functions can vary.

TODO : Can implement more aggregators
"""
class MeanAggregator(nn.Module):
    """
    Aggregates a node's embedding using mean of neighbors' embeddings.
    """
    def __init__(self,features,cuda=False,verbose=False,
                    gcn=False):
        """
        Inits an aggregator for this graph.
        features -- A function mapping a long tensor of node ids to 
                    a float tensor of feature values.
        cuda     -- Whether to cast tensors in this scope to the GPU
        """
        super(MeanAggregator,self).__init__()

        self.features = features
        self.cuda = cuda
        self.verbose = verbose
        self.gcn = gcn

    def forward(self, nodes, neighbours, num_sample=5):
        """
        The aggregator needs:
        nodes -- list of nodes in a batch
        neighbours -- list of sets, i-th set is i-th nodes neighbours.
        num_sample -- n_samples to draw from the neighbourhood
                   ^> None means use full neighbourhood!
        """
        # Making these pointers makes them faster 
        _set = set
        _sample = random.sample 
        # First let's find neighbours we need to sample
        # We sample a distinct set for each node.
        if num_sample is not None: # if performing sampling
            samp_neighs = []
            for neighbourhood in neighbours:
                if len(neighbourhood) >= num_sample:
                    # If there are more neighbours than samples in a
                    # neighbourhood we sample it
                    samp_neighs.append(_set(_sample(neighbourhood,num_sample)))
                else: # If less then we just take as is
                    samp_neighs.append(neighbourhood)
        else: 
            samp_neighs = neighbours 
        # Add self loops 
        if self.gcn:
            samp_neighs = [samp_neigh + set([nodes[i]]) for i, samp_neigh in
                           enumerate(samp_neighs)]
        # Find all unique nodes
        unique_nodes_list = list(set.union(*samp_neighs)) 
        # Assign all current nodes an index accessed by their node_id (n) 
        # in a dict.
        unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
        # Create a mask which will allow the layer to only show data from connected
        # nodes to other connected nodes. This has an index for each node in the
        # batch with an index for each unique node it may be connected to
        mask = torch.zeros(len(samp_neighs),len(unique_nodes))
        # Next we find the indices of the maks for the edges
        # (This is essentially a condensed adjacency matrix)
        column_indices = []
        row_indices = []
        # samp_neighs is an list of sets of indices representing edges
        for i,connected_nodes in enumerate(samp_neighs):
            n_neighs = len(connected_nodes)
            for n in connected_nodes:
                # Network Ordering -> Batch Ordering
                column_indices.append(unique_nodes[n])
            # Associated indices
            for j in range(n_neighs):
                row_indices.append(i)
        # Set the value of these nodes to 1
        mask[row_indices,column_indices] = 1
        # Send to GPU if needed
        if self.cuda:
            mask = mask.cuda()
        # Calculate the number of neigbours for each node
        num_neigh = mask.sum(1,keepdim=True)
        # Divide each 1 in the mask by the number of neighbours
        # This is a step towards calculating the mean!
        mask = mask.div(num_neigh)
        # Get the features of all nodes involved:
        # !!! This call recursively calls all previous aggregators in the
        # !!! training process
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        # Matrix multiplication here produces the aggregate averaged features
        to_feats = mask.mm(embed_matrix.float())
        return to_feats

class Encoder(nn.Module):
    """
    Encodes nodes.

    Feature_dim can be seen as input shape.
    Embed_dim can be seen as output shape.
    Features is a function that returns a representation of a node
    """ 
    def __init__(self, features, feature_dim, 
            embed_dim, adj_lists, aggregator,
            num_sample=10,
            base_model=None, cuda=False, 
            feature_transform=False,verbose=False): 
        super(Encoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        self.verbose = verbose
        self.activation = "relu" # TODO : Implement other functions
        if base_model != None:
            self.base_model = base_model
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        # The weight is the learnt parameter of this layer used to highlight
        # the important regions of the incoming representation.
        # It is 2 x feat_dim as it uses two representations at once, the local
        # and neighbourhood aggregated.
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, 2 * self.feat_dim))
        init.xavier_uniform_(self.weight)

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        nodes     -- list of nodes
        """
        # Calculate aggregated representations
        neigh_feats = self.aggregator.forward(nodes, 
                                              [self.adj_lists[int(node)] for 
                                               node in nodes], 
                                              self.num_sample)
        # Cast these to GPU if necessary
        if self.cuda:
            self_feats = self.features(torch.LongTensor(nodes).cuda())
        else:
            self_feats = self.features(torch.LongTensor([*nodes]))
        # Concatenate the layers previous representation with current
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        # Feed this through an activation function
        if self.activation == "relu":
            combined = F.relu(self.weight.mm(combined.t().float()))
        return combined

class GraphSAGEEncoder(nn.Module):
    """
    Encodes nodes.

    Feature_dim can be seen as input shape.
    Embed_dim can be seen as output shape.
    Features is a function that returns a representation of a node
    """ 
    def __init__(self, features, feature_dim, 
            embed_dim, adj_lists, aggregator,
            num_sample=10,
            base_model=None, cuda=False, 
            feature_transform=False,verbose=False): 
        super(Encoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        self.verbose = verbose
        self.activation = "relu" # TODO : Implement other functions
        if base_model != None:
            self.base_model = base_model
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, 2 * self.feat_dim))
        init.xavier_uniform_(self.weight)

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        nodes     -- list of nodes.

        In the graph sage implementation they use a maxpooling on the
        neighbourhood state along with including the addition of the aggregated
        state into the activation function input.

        TODO: Implement max pooling.
        """
        # Calculate aggregated representations
        neigh_feats = self.aggregator.forward(nodes, 
                                              [self.adj_lists[int(node)] for 
                                               node in nodes], 
                                              self.num_sample)
        # Cast these to GPU if necessary
        if self.cuda:
            self_feats = self.features(torch.LongTensor(nodes).cuda())
        else:
            self_feats = self.features(torch.LongTensor([*nodes]))
        # Concatenate the layers previous representation with current
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        # Feed this through an activation function
        if self.activation == "relu":
            combined = F.relu(self.weight.mm(combined.t().float())+neigh_feats)
        return combined
"""
Unit tests!
"""
if __name__ == '__main__':
    adj, degrees, feats = featureless_random_graph(100)
    adj_list = get_neigh_list(adj)
    node_list = [*range(adj.shape[0])]
    features = get_feature_func(node_list,feats)
    test_agg = MeanAggregator(features,cuda=True)
    test_layer = Encoder(features, 
                         feats.shape[1],
                         5, 
                         adj_list,
                         test_agg,
                        num_sample=None)
    # Attempt to feed a graph through the layer
    for i in range(len(adj_list)):
        n_con = len(adj_list[i])
        print("n_connections:", n_con)
        print(adj_list[i])
        if n_con == 0:
            print("No connections! Skipping...")
            continue
        x = test_layer(adj_list[i]) # Feed this a node neighbour list
        print(f"Success for {i}!")
        print(x.shape)
        print()


ModuleNotFoundError: No module named 'torch._C'

In [ ]:
#modules.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from layers import MeanAggregator, Encoder
from utils import featureless_random_graph,get_feature_func,get_neigh_list
"""
To begin with let's try to implement the GAP-scalefree model
which uses GraphSAGE layers.
Training it on undirected graphs first let's us simplify the model
so we will do the GAP-scalefree model on a E-R produced graph
"""
class GraphEmbeddingModule(nn.Module):
    """
    This part of the network takes in a graph G = (V,E) and 
    produces node embeddings of each node in the network using 
    classical GraphSAGE layers.
    We first implement a 4 layers of GraphSAGE with 128 hidden units.
    """
    def __init__(self,
                features,
                feature_dim,
                adj_lists,
                embed_dim=64,
                hidden_dim=128,
                num_sample=2,
                cuda=False):
        """
        Note that the features as aggregated by the last layer are passed to the next
        layer using a lambda function.
        This means when we call the final layer, it will recursively call the
        preceding layers.
        """
        # TODO : This can be neatened up a lot!
        super().__init__()
        self.agg1 = MeanAggregator(features,cuda=cuda)
        self.gSAGE1 = Encoder(features,
                              feature_dim,
                              hidden_dim,
                              adj_lists,
                              self.agg1,
                              num_sample=num_sample,
                              cuda=cuda)
        self.agg2 = MeanAggregator(lambda nodes : self.gSAGE1(nodes).t(),
                                   cuda=cuda,verbose=True)
        self.gSAGE2 = Encoder(lambda nodes : self.gSAGE1(nodes).t(),
                              hidden_dim,
                              hidden_dim,
                              adj_lists,
                              self.agg2,
                              num_sample=num_sample,
                              cuda=cuda,
                              base_model=self.gSAGE1,
                             verbose=True)
        self.agg3 = MeanAggregator(lambda nodes : self.gSAGE2(nodes).t(),cuda=cuda)
        self.gSAGE3 = Encoder(lambda nodes : self.gSAGE2(nodes).t(),
                              hidden_dim,
                              hidden_dim,
                              adj_lists,
                              self.agg3,
                              num_sample=num_sample,
                              cuda=cuda,
                             base_model=self.gSAGE2)
        self.agg4 = MeanAggregator(lambda nodes : self.gSAGE3(nodes).t(),cuda=cuda)
        self.gSAGE4 = Encoder(lambda nodes : self.gSAGE3(nodes).t(),
                              hidden_dim,
                              embed_dim,
                              adj_lists,
                              self.agg4,
                              num_sample=num_sample,
                              cuda=cuda,
                             base_model=self.gSAGE3)
        
    def forward(self,n):
       """
        Here we only need to call the last layer to call all!
       """
       embeddings = self.gSAGE4(n)
       return embeddings.t()

    def change_graph(self,new_adj,new_features):
       """
        Helper method to specialise to different graphs
       """
       self.gSAGE1.adj_lists = new_adj
       self.gSAGE2.adj_lists = new_adj
       self.gSAGE3.adj_lists = new_adj
       self.gSAGE4.adj_lists = new_adj

       self.gSAGE1.features = new_features
       self.agg1.features = new_features


class GraphPartitioningModule(nn.Module):
    """
    This part of the network takes in the node embeddings
    and produces probabilities of the node belonging to each
    state.
    Roughly a transform from
    [n_nodes, embedding_space_dim] --> [n_nodes,p_per_partition]
    """
    def __init__(self,latent_dim,n_partitions):
        super().__init__()
        self.dense = nn.Linear(latent_dim,n_partitions)
    
    def forward(self,embeddings):
        embeddings = self.dense(embeddings)
        return F.softmax(embeddings,dim=1)

                            
""" Unit tests! """
if __name__ == '__main__':
    adj, degrees, feats = featureless_random_graph(100)
    adj_list = get_neigh_list(adj)
    node_list = [*range(adj.shape[0])]
    input_nodes = [*range(adj.shape[0])]
    to_remove = []
    for i in input_nodes:
        if len(adj_list[i]) == 0:
            print(f"Disconnected Node {i}")
            to_remove.append(i)
    for i in to_remove[::-1]:
        input_nodes.pop(i)
    features = get_feature_func(node_list,feats)
    test_E = GraphEmbeddingModule(features,
                      feats.shape[1],
                      adj_list,
                      cuda=False)
    ## Let's embed a whole graph as a test
    x = test_E(input_nodes)

    # And feed this to a Graph Partition module
    n_categories = 5
    latent = 5
    test_G = GraphPartitioningModule(latent,n_categories)
    prob = test_G(x)


In [ ]:
#train.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import networkx as nx
import os
from tqdm import tqdm
from modules import GraphEmbeddingModule, GraphPartitioningModule
from utils import *
from datetime import datetime
class GAPmodel(nn.Module):
    """
    The full model.
    Simply links the two modules together
    """
    def __init__(self,
                features,
                feature_dim,
                adj_lists,
                n_partitions=2,
                embed_dim=64,
                hidden_dim=128,
                num_sample=10,
                cuda=True):
        super().__init__()
        self.embedding = GraphEmbeddingModule(
                            features,
                            feature_dim,
                            adj_lists,
                            embed_dim=embed_dim,
                            hidden_dim=hidden_dim,
                            num_sample=num_sample,
                            cuda=cuda
                        )
        self.partitioning = GraphPartitioningModule(
                            embed_dim,
                            n_partitions
                        )
    def forward(self,nodes):
        nodes = self.embedding(nodes)
        probability = self.partitioning(nodes)
        return probability

def expected_cut_loss(probabilities,
                      degree,
                      adj_matrix):
    if not isinstance(adj_matrix, torch.Tensor):
        adj_matrix = torch.tensor([[*a] for a in adj_matrix])
    gamma = probabilities.t() @ degree
    yt = (1-probabilities).t()
    loss = (probabilities.div(gamma.t())) 
    loss = loss @ yt
    loss = torch.sum(loss * adj_matrix)
    return loss

if __name__ == '__main__':
    date = datetime.now()
    run_name = date.strftime("%b_%d_%H_%M") + "_run"
    # Training variables
    # TODO : Move into argparser
    n_graphs = 1   # n_unique graphs
    n_nodes = 100 # n_nodes in graph
    n_times = 100   # n_times exposed to each graph
    epoch_gen = tqdm(range(n_graphs))
    # Gen the first graph
    input_nodes, adj_list, features, degree, adj = gen_graph(n_nodes,run_name,0)
    features_inp = features(0).shape[0]
    # Init the model
    model = GAPmodel(features,
                      features_inp,
                      adj_list,
                      cuda=False)
    # Assign the optimizer
    optimizer = optim.Adam(model.parameters(),lr=7.5e-6)
    # Begin Training
    model.train()
    for i in epoch_gen:
        if i != 0:
            # Generate a new random graph
            input_nodes, adj_list, features, degree, adj = gen_graph(n_nodes,
                                                                     run_name,i)
            # Assign the model new features and adjacency
            model.embedding.change_graph(adj_list,features)
        for k in range(n_times):
            optimizer.zero_grad()
            node_probs = model(input_nodes)
            loss = expected_cut_loss(node_probs,degree[:,1],adj)
            loss.backward()
            optimizer.step()
            print("LOSS FOR THIS: ",loss.item())
    with torch.no_grad():
        image_graph(adj,run_name,i,predictions=node_probs)
        ec, balance = rate_preds(adj,node_probs)
        print("EDGE CUT :",ec,"BALANCE :",balance)
        with open("./graphs/scores.txt","a") as f:
            f.write(f"{run_name} {n_graphs} {n_nodes} {n_times} {ec} {balance}\n")


In [11]:
#utils.py conversion

import torch
import os
import networkx as nx
from matplotlib import pyplot as plt
from networkx.generators import erdos_renyi_graph as er_graph
from networkx.generators import scale_free_graph as sfg
from networkx.linalg import adjacency_matrix
from sklearn.decomposition import PCA
from collections import Counter

"""
Here we will: 
- Preprocess generated graphs
- Implement methods to measure results
"""
def featureless_random_graph(n_nodes,prob=0.1,
                kind="er",
                feature_size=10):
    """
    Generates a random graph and then performs pca
    on it and stores this as a feature matrix.
    Returns: 
      -->   adjacency matrix    [n_nodes,n_nodes]
      -->   node degree matrix  [n_nodes,1]
      -->   node feature matrix [n_nodes,feature_size] 
      in that order.
    Kind is either sf: Scale Free
                   er: Erdos-Renyi
    Features size :int: is the dimension of PCA to return.
    """
    # Generate graph
    if kind == "sf":
        G = sfg(n_nodes)
    elif kind == "er":
        G = er_graph(n_nodes, prob)
    else:
        print(f"Kind: {kind} not understood")
        raise Exception

    # Generate adjacency matrix
    adjacency_m = adjacency_matrix(G) # This returns a SPARSE matrix!
    adjacency_m = adjacency_m.toarray()
    adjacency_m = torch.tensor(adjacency_m)

    # Generate node degree matrix
    node_degrees = [*G.degree()]
    node_degrees = torch.tensor(node_degrees).float()

    # Generate node features
    pca = PCA(n_components=feature_size)
    node_features = pca.fit_transform(adjacency_m)
    node_features = torch.tensor(node_features)

    return adjacency_m, node_degrees, node_features

# Utils
def get_feature_func(nodes,features):
    """
    Helper function to allow the node to call initial features like it would a
    layer output.
    """
    def feature_func(node):
        return features[node]
    return feature_func

def get_neigh_list(adj_m,to_set=True):
    adj_list = []
    for node in adj_m.tolist():
        neigh_list = []
        for neigh_id in range(len(node)):
            if node[neigh_id] != 0:
                if not to_set:
                    for j in range(node[neigh_id]):
                        neigh_list.append(neigh_id)
                else:
                    neigh_list.append(neigh_id)
        adj_list.append(set(neigh_list))
    return adj_list

def image_graph(adjacency_matrix,run_name,i,predictions=None):
    G = nx.Graph(adjacency_matrix.detach().numpy())
    path = f"./graphs/{run_name}/"
    if predictions is not None:
        test_labels = gen_test_labels(predictions)
    else: 
        test_labels = None
    if not os.path.isdir(path):
        os.makedirs(path)
    plt.figure(figsize=(12,12))
    nx.draw(G,node_color=test_labels,with_labels=True)
    plt.savefig(path+str(i)+"-result.png")
    plt.close()

def gen_graph(n_nodes,run_name,epoch,save=True):
    # Generate a random graph
    adj, degrees, feats = featureless_random_graph(n_nodes)
    adj_list = get_neigh_list(adj)
    node_list = [*range(adj.shape[0])]
    input_nodes = [*range(adj.shape[0])]
    features = get_feature_func(node_list,feats)
    if save and run_name:
        path = f"./graphs/{run_name}/"
        if not os.path.isdir(path):
            os.makedirs(path)
        plt.figure(figsize=(12,12))
        nx.draw(nx.Graph(adj.detach().numpy()),with_labels=True)
        plt.savefig(path + str(epoch) + ".png")
        plt.close()
    return input_nodes, adj_list, features, degrees, adj

def gen_test_labels(probabilities):
    test_labels = [torch.argmax(p).item() for p in probabilities]
    colour = []
    for p in test_labels:
        if p == 0:
            colour.append("blue")
        if p == 1:
            colour.append("red")
        if p == 2:
            colour.append("yellow")
        if p == 3:
            colour.append("olive")
        if p == 4:
            colour.append("cyan")
    return colour

def rate_preds(adj,preds):
    n_nodes = adj.shape[0]
    # Take the predicted state
    states = [max(p) for p in preds]
    # Get edge-list
    edges = adj.nonzero(as_tuple=False).numpy()
    # Calc edge cut
    edge_cut = calc_edge_cut(edges,states)
    # Calc balance
    balance = calc_balance(states)
    return edge_cut, balance

def calc_edge_cut(edges,states):
    """
    Edge cut is the ratio of the cut to the number of edges.
    edges should be a iterable of iterables of edge indices.
    states should be a list of the predicted states
    """
    n_edges = len(edges)
    states = {i:s for i,s in enumerate(states)}
    cuts = 0
    for (state1,state2) in edges:
        if states[state1] != states[state2]:
            cuts += 1
    return cuts / n_edges


def calc_balance(states):
    """
    From what I can gather:
    B = 1 - MSE(node_populations,n_nodes/n_parts)
    B = 1 - 1/n*sum(node_pop - n_nodes/n_parts)**2
    """
    n_nodes = len(states)
    node_pops = Counter(states)
    balance = n_nodes / len(node_pops.keys())
    mse = sum((v - balance)**2 for v in node_pops.values()) 
    mse *= 1 / n_nodes
    return 1 - mse

if __name__ == '__main__':
    """
    Unit tests!
    """
    adj,degrees,features = featureless_random_graph(100)
    print("Adjacency: ", adj.shape)
    print("Degrees: " ,  degrees.shape)
    print("Features: ",  features.shape)

    # Getting into right format for layers
    adj_list = get_neigh_list(adj)
    node_list = [*range(adj.shape[0])]
    features = get_feature_func(node_list,features)


Adjacency:  torch.Size([100, 100])
Degrees:  torch.Size([100, 2])
Features:  torch.Size([100, 10])


In [22]:
import scipy as sp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
from os.path import dirname, join as pjoin
import scipy.io as sio
from matplotlib import pyplot as plt
from networkx.generators import erdos_renyi_graph as er_graph
from networkx.generators import scale_free_graph as sfg
from networkx.linalg import adjacency_matrix
from sklearn.decomposition import PCA
from collections import Counter
import torch

#ADJ MATRIX FROM MATLAB

Adj = np.loadtxt(r'M:\Masters\Dynamical Coarse Graining in Complex Networks\Adj.txt')

def get_neigh_list(adj_m,to_set=True):
    adj_list = []
    for node in adj_m.tolist():
        neigh_list = []
        for neigh_id in range(len(node)):
            if node[neigh_id] != 0:
                if not to_set:
                    for j in range(node[neigh_id]):
                        neigh_list.append(neigh_id)
                else:
                    neigh_list.append(neigh_id)
        adj_list.append(set(neigh_list))
    return adj_list

def image_graph(adjacency_matrix,run_name,i,predictions=None):
    G = nx.Graph(adjacency_matrix.detach().numpy())
    path = f"./graphs/{run_name}/"
    if predictions is not None:
        test_labels = gen_test_labels(predictions)
    else: 
        test_labels = None
    if not os.path.isdir(path):
        os.makedirs(path)
    plt.figure(figsize=(12,12))
    nx.draw(G,node_color=test_labels,with_labels=True)
    plt.savefig(path+str(i)+"-result.png")
    plt.close()

adj_list = get_neigh_list(Adj)
print(adj_list)

Adj = torch.tensor(Adj)
image_graph(Adj, 1, 100)
print(Adj)

[{2, 5, 6, 10, 15, 16, 21}, {3, 5, 7, 12, 13, 15, 19, 20, 21, 23}, {0, 4, 7, 13, 14, 16, 18, 19, 20, 21, 22, 23}, {1, 6, 8, 9, 11, 15, 22, 24}, {2, 5, 6, 7, 8, 9, 10, 11, 17, 18, 19, 22, 61}, {0, 1, 4, 8, 10, 13, 18, 19, 20}, {0, 3, 4, 10, 12, 13, 14, 17, 19, 20, 29}, {1, 2, 4, 9, 16, 17, 18, 19, 21, 22, 23}, {3, 4, 5, 15, 18, 19, 21, 24}, {3, 4, 7, 10, 13, 14, 15, 16, 17, 21, 23, 24}, {0, 4, 5, 6, 9, 14, 17, 19, 21, 22, 23, 24}, {3, 4, 12, 14, 17, 19}, {1, 6, 11, 15, 16, 17, 19, 24}, {1, 2, 5, 6, 9, 14, 15, 16, 17, 18, 20, 22, 23, 24}, {2, 6, 9, 10, 11, 13, 15, 17, 23}, {0, 1, 3, 8, 9, 12, 13, 14, 16, 17, 82, 24}, {0, 2, 7, 9, 12, 13, 15, 50, 52, 23, 24}, {34, 4, 6, 7, 9, 10, 11, 12, 13, 14, 15, 18, 19, 21, 23, 24}, {2, 4, 5, 7, 8, 13, 17, 19, 22}, {1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 17, 18, 22}, {1, 2, 5, 6, 13, 45, 21}, {0, 1, 2, 7, 8, 9, 10, 17, 20, 22}, {2, 3, 4, 7, 10, 13, 79, 18, 19, 21, 26}, {1, 2, 7, 9, 10, 13, 14, 16, 17, 24}, {3, 8, 9, 10, 12, 13, 15, 16, 17, 23}, {33, 34, 35,

In [17]:
#Generating new graph

random_graph = featureless_random_graph(100)
print(random_graph)

adj_list = get_neigh_list(adjacency_m)

(tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([[ 0.,  5.],
        [ 1., 12.],
        [ 2., 12.],
        [ 3., 12.],
        [ 4., 13.],
        [ 5., 16.],
        [ 6., 12.],
        [ 7., 11.],
        [ 8., 10.],
        [ 9.,  7.],
        [10.,  9.],
        [11.,  6.],
        [12., 10.],
        [13., 12.],
        [14.,  7.],
        [15., 13.],
        [16.,  5.],
        [17., 15.],
        [18., 10.],
        [19.,  5.],
        [20., 11.],
        [21., 12.],
        [22., 13.],
        [23., 11.],
        [24., 11.],
        [25., 10.],
        [26.,  7.],
        [27., 19.],
        [28.,  8.],
        [29., 11.],
        [30.,  4.],
        [31.,  8.],
        [32.,  8.],
        [33., 10.],
        [34., 10.],
        [35., 10.],
        [36., 14.],
        [37.,  8.],
 

NameError: name 'adjacency_m' is not defined

In [20]:
import utils
import networkx as nx

d = utils.featureless_random_graph(100)
print(d[1])

tensor([[ 0.,  9.],
        [ 1.,  9.],
        [ 2., 10.],
        [ 3.,  7.],
        [ 4., 11.],
        [ 5.,  5.],
        [ 6., 11.],
        [ 7.,  9.],
        [ 8., 11.],
        [ 9., 12.],
        [10., 15.],
        [11.,  9.],
        [12.,  7.],
        [13.,  7.],
        [14., 14.],
        [15., 10.],
        [16.,  7.],
        [17., 11.],
        [18.,  7.],
        [19.,  8.],
        [20.,  9.],
        [21., 14.],
        [22., 10.],
        [23., 11.],
        [24.,  9.],
        [25.,  8.],
        [26.,  6.],
        [27., 13.],
        [28., 11.],
        [29., 13.],
        [30., 12.],
        [31.,  8.],
        [32., 11.],
        [33., 12.],
        [34., 15.],
        [35., 12.],
        [36.,  8.],
        [37., 10.],
        [38., 13.],
        [39., 10.],
        [40., 16.],
        [41., 13.],
        [42.,  6.],
        [43.,  4.],
        [44.,  7.],
        [45., 11.],
        [46., 10.],
        [47., 10.],
        [48.,  5.],
        [49., 10.],
